In [1]:
import torch
import matplotlib.pyplot as plt
import plotly.express as px
import sys
import glob
import json
import numpy as np
from tqdm import tqdm
sys.path.append("../")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
norms  = np.load("/app/dataset/various_splits/all_layout/normalizers.npy",allow_pickle=True).item()

In [136]:
path="/app/alexnet_train_batch_32.npz"

In [137]:
opcode_name = json.load(open("/app/nn-runtime-network/assets/opt_code.json"))
feature_vec_name = json.load(open("/app/nn-runtime-network/assets/node_feature_vector.json"))

In [138]:
import glob
import os
import pandas as pd

def get_npy_files(root_path):
    # Use a glob pattern to match all .npz files in the directory and subdirectories
    pattern = os.path.join(root_path, '**', '*.npz')
    npy_files = glob.glob(pattern, recursive=True)
    return npy_files

all_npy_files = get_npy_files("/app/dataset/npz_all/npz/layout")+get_npy_files("/app/dataset/npz_all/npz/tile/xla")
print(len(all_npy_files))
# Create a DataFrame from the list of npy files
df = pd.DataFrame(all_npy_files, columns=["path"])
print(len(df))
def find_split(x):
    if "/train" in x:
        return "train"
    elif "/valid" in x:
        return "valid"
    elif "/test" in x:
        return "test"
    return None
def find_configtype(x):
    if "/default" in x:
        return "default"
    elif "/random" in x:
        return "random"
    return "istile"
def find_modeltype(x):
    if "/nlp" in x:
        return "nlp"
    elif "/xla" in x:
        return "xla"
    return None
def find_grapthtype(x):
    if "/layout" in x:
        return "layout"
    elif "/tile" in x:
        return "tile"
    return None
df["split"] = df.path.apply(lambda x : find_split(x))
df["config_type"] = df.path.apply(lambda x : find_configtype(x))
df["modeltype"] = df.path.apply(lambda x : find_modeltype(x))
df["graphtype"] = df.path.apply(lambda x : find_grapthtype(x))
df["modelname"] = df.path.apply(lambda x: x.split("/")[-1])

7868
7868


In [145]:
path = df[(df.modeltype=="nlp")&(df.graphtype=="layout")&(df.split=="train")].path.to_list()[0]

In [146]:
data = dict(np.load(path))

In [147]:
def node_config_feat_string(inf):
    outstr = "x".join([str(int(i)) for i in inf[:6]])
    inpstr = "x".join([str(int(i)) for i in inf[6:12]])
    kernstr = "x".join([str(int(i)) for i in inf[12:]])
    return f"out: {outstr},inp: {inpstr}, kern: {kernstr}"
def node_feat_string(inf):
    layoutminmaj = "x".join([str(int(i)) for i in inf[134:]])
    shapdim = "x".join([str(int(i)) for i in inf[21:27] ])
    dimensions = "x".join([str(int(i)) for i in inf[31:37] ])
    convdims = "x".join([str(int(i)) for i in inf[93:106]])
    return f"lay:{layoutminmaj},shape:{shapdim},dims:{dimensions}\n convdim:{convdims}"

In [148]:
import matplotlib.pyplot as plt
import networkx as nx
from pyvis.network import Network
import numpy as np
import matplotlib

# Initialize a directed networkx graph
G = nx.DiGraph()

# Add nodes with attributes to the graph
for idx, opcode in enumerate(data['node_opcode']):
    G.add_node(int(idx), opcode=int(opcode))

# Add directed edges to the graph
edges = data['edge_index']
for edge in edges:
    G.add_edge(int(edge[1]), int(edge[0]))


nodes_to_keep = list(nx.bfs_tree(G, source=0, depth_limit=999))  # This assumes node IDs start at 0

nodes_to_keep = nodes_to_keep[:1000]

G = G.subgraph(nodes_to_keep).copy()

nt = Network("800px", "100%", notebook=False, heading='Directed Graph Visualization', directed=True)

# Create a colormap with matplotlib
colormap = plt.get_cmap('nipy_spectral')
norm = plt.Normalize(min(data['node_opcode']), max(data['node_opcode']))

opcode_colors = {opcode: colormap(norm(opcode)) for opcode in data['node_opcode']}

# Take the networkx graph and translate it to a pyvis graph
nt.from_nx(G)
nodeconf_ids =  data["node_config_ids"].tolist()
# Set options for nodes
for node in nt.nodes:
    opcode = G.nodes[node['id']]['opcode']
    nodestring = f"Opcode: {opcode_name[str(opcode)]}"

    color = matplotlib.colors.rgb2hex(opcode_colors[opcode])  # Convert RGBA to hex
    if node["id"] in nodeconf_ids:
        idx = nodeconf_ids.index(node["id"])
        nodestring = nodestring+"\n conf1>"+node_config_feat_string(data["node_config_feat"][0,idx])
        nodestring = nodestring+"\n conf2>"+node_config_feat_string(data["node_config_feat"][len(data["node_config_feat"])//2,idx])
        nodestring = nodestring+"\n conf3>"+node_config_feat_string(data["node_config_feat"][len(data["node_config_feat"])-1,idx])
    nodestring = nodestring+ "\n node>"+node_feat_string(data["node_feat"][node["id"]])
    node["title"] = nodestring
    node["label"] = ''  # You can remove or set to some label you want to show
    node["color"] = color
    node["size"] = 10 if opcode not in [2,26,75,34,63] else 20
    node["value"] = node["size"] 

# Set the physics layout and interaction options of the network
nt.set_options("""
var options = {
  "nodes": {
    "font": {
      "size": 12
    }
  },
  "edges": {
    "color": {
      "inherit": true
    },
    "arrows": {
      "to": { "enabled": true, "scaleFactor": 0.5 }
    },
    "smooth": false
  },
  "physics": {
    "forceAtlas2Based": {
      "gravitationalConstant": -26,
      "centralGravity": 0.005,
      "springLength": 230,
      "springConstant": 0.18
    },
    "maxVelocity": 146,
    "solver": "forceAtlas2Based",
    "timestep": 0.35,
    "stabilization": { "iterations": 150 }
  }
}
""")

# Save the graph to an HTML file
nt.write_html('graph.html')


In [135]:
path

'/app/dataset/npz_all/npz/layout/xla/default/train/alexnet_train_batch_32.npz'